In [1]:
%%html
<style>
table {float:left}
</style>

# **Working with HPE IMC API for Custom Views**

In this notebook, we will be covering the basics of using the pyhpimc python module to access the RESTFUL interface ( *eAPI* ) of the HPE IMC Network Management Server. 

The python library is currently available at [HPE Github repository](https://github.com/HPENetworking/HP-Intelligent-Management-Center/blob/master/README.md)

## Import Required Modules from pyhpeimc
Before we get started, we must import the required libraries into the ipython interpreter . For this notebook, you will require the Auth and Operator modules from the pyhpimc library, as well as the standard csv module. 

In [7]:
import csv
import time
from pyhpeimc.auth import *
from pyhpeimc.plat.groups import *
from pyhpeimc.version import *

In [8]:
2+34

36

## Input IMC Credentials

In this section, we will first create a authentication object which will contain

- the protocol
- ip address
- port number
- username
- password

If you are running this on your local network, please change the appropriate values in the following field to connect to your local IMC server. 

In [10]:
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")


## Create Print View List Helper Function

This function will gather the current list of custom views from the HPE IMC NMS and print them out to the screen.

In [11]:
def print_views():
    views_list = get_custom_views(url=auth.url, auth=auth.creds)
    print ("There are a total of " + str(len(views_list)) + " views currently")
    for view in views_list:
        print (view['name'])
    print (json.dumps(views_list[0], indent = 4))

## Display the current views

This section will count the total number of existing views as well as display the first view returned.

Notice the 'upLevelSymbolId" is **3** for this custom view. The three designates that this is a level one custom view, meaning that it's at the root of the custom view tree.

In [12]:
print_views()

There are a total of 103 views currently
Branch1
Branch10
Branch100
Branch11
Branch12
Branch13
Branch14
Branch15
Branch16
Branch17
Branch18
Branch19
Branch2
Branch20
Branch21
Branch22
Branch23
Branch24
Branch25
Branch26
Branch27
Branch28
Branch29
Branch3
Branch30
Branch31
Branch32
Branch33
Branch34
Branch35
Branch36
Branch37
Branch38
Branch39
Branch4
Branch40
Branch41
Branch42
Branch43
Branch44
Branch45
Branch46
Branch47
Branch48
Branch49
Branch5
Branch50
Branch51
Branch52
Branch53
Branch54
Branch55
Branch56
Branch57
Branch58
Branch59
Branch6
Branch60
Branch61
Branch62
Branch63
Branch64
Branch65
Branch66
Branch67
Branch68
Branch69
Branch7
Branch70
Branch71
Branch72
Branch73
Branch74
Branch75
Branch76
Branch77
Branch78
Branch79
Branch8
Branch80
Branch81
Branch82
Branch83
Branch84
Branch85
Branch86
Branch87
Branch88
Branch89
Branch9
Branch90
Branch91
Branch92
Branch93
Branch94
Branch95
Branch96
Branch97
Branch98
Branch99
Branches
My Network View
WAN
{
    "statusDesc": "Unmanaged",
    "

![Default Custom Views](./images/customviews1.png "Default Custom Views")

## Create New Custom View

In this step, we will create two custom views **Canada** and **Alberta** using the create_custom_views() function.

For this example, we will create 
- the **Canada** view which is a L1 custom view
- the **Alberta** custom view which will be a child of the **Canada** Custom view.
- the **Calgary** custom view which will be a child of the **Alberta** Custom View

|Name| Upperview |
|:-----|-----|
|Canada|     |
|Alberta| Canada |
|Calgary | Alberta |

In [25]:
create_custom_views(auth=auth.creds, url=auth.url, name="Canada")
create_custom_views(auth=auth.creds, url=auth.url, name="Alberta",upperview='Canada')
create_custom_views(auth=auth.creds, url=auth.url, name="Calgary",upperview='Alberta')


print_views()

There are a total of 4 views currently
Alberta
Calgary
Canada
My Network View
{
    "name": "Alberta",
    "runStatus": "-1",
    "statusImgSrc": "res/icon_custom_view_unmanaged_16x16.png",
    "upLevelSymbolId": "2828",
    "symbolId": "2829",
    "statusDesc": "Unmanaged"
}


![Manual Custom Views](./images/customviews2.png "Manual Custom Views")

### Canada View

Notice that the **upLevelSymbolId** for this view is the same as the default **My Network View** shown above. We can see that this is a L1 custom view. 

In [8]:
get_custom_views(url=auth.url, auth=auth.creds, name="Canada")

[{'name': 'Canada',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1862'}]

### Alberta View

Notice that **upLevelSymbolId** for this view is equal to the **symbolId** of the **Canada** view shown in the previous example. we can see that this is a child of the Canada Custom View.

In [9]:
get_custom_views(url=auth.url, auth=auth.creds, name="Alberta")

[{'name': 'Alberta',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1863'}]

### Calgary View

Notice that **upLevelSymbolId** for this view is equal to the **symbolId** of the **Alberta** view shown in the previous example. we can see that this is a child of the Canada Custom View.

In [10]:
get_custom_views(url=auth.url, auth=auth.creds, name="Calgary")

[{'name': 'Calgary',
  'runStatus': '-1',
  'statusDesc': 'Unmanaged',
  'statusImgSrc': 'res/icon_custom_view_unmanaged_16x16.png',
  'symbolId': '1864'}]

## Deleting Custom Views

In this section we will delete individual custom views.

In [26]:
delete_custom_view(url=auth.url, auth=auth.creds, name='Canada')
delete_custom_view(url=auth.url, auth=auth.creds, name='Alberta')
delete_custom_view(url=auth.url, auth=auth.creds, name='Calgary')
print_views()

There are a total of 1 views currently
My Network View
{
    "name": "My Network View",
    "runStatus": "5",
    "statusImgSrc": "res/icon_custom_view_critical_16x16.png",
    "upLevelSymbolId": "3",
    "symbolId": "2777",
    "statusDesc": "Critical"
}


![Deleted Custom Views](./images/customviews4.png "Deleted Custom Views")

# Display Contents of Custom Views.csv

We have prepared a CSV file which contains the new custom views that we wish to create. This can contain both parent and child custom views. 

| name | upperview |
| ------ | ----------- |
| Branches   |   |
| Branch1 | Branches |
| Branch2    | Branches |
| WAN      |            |

In [4]:
with open('custom_views.csv') as f:
    s = f.read()
    print (s)

name,upperview
Branches,
Branch1,Branches
Branch2,Branches
WAN,
Branch3,
Branch4,
Branch5,
Branch6,
Branch7,
Branch8,
Branch9,
Branch10,
Branch11,
Branch12,
Branch13,
Branch14,
Branch15,
Branch16,
Branch17,
Branch18,
Branch19,
Branch20,
Branch21,
Branch22,
Branch23,
Branch24,
Branch25,
Branch26,
Branch27,
Branch28,
Branch29,
Branch30,
Branch31,
Branch32,
Branch33,
Branch34,
Branch35,
Branch36,
Branch37,
Branch38,
Branch39,
Branch40,
Branch41,
Branch42,
Branch43,
Branch44,
Branch45,
Branch46,
Branch47,
Branch48,
Branch49,
Branch50,
Branch51,
Branch52,
Branch53,
Branch54,
Branch55,
Branch56,
Branch57,
Branch58,
Branch59,
Branch60,
Branch61,
Branch62,
Branch63,
Branch64,
Branch65,
Branch66,
Branch67,
Branch68,
Branch69,
Branch70,
Branch71,
Branch72,
Branch73,
Branch74,
Branch75,
Branch76,
Branch77,
Branch78,
Branch79,
Branch80,
Branch81,
Branch82,
Branch83,
Branch84,
Branch85,
Branch86,
Branch87,
Branch88,
Branch89,
Branch90,
Branch91,
Branch92,
Branch93,
Branch94,
Branch95,
Branch96,
Bra

# Create Import Custom Views Function

Here we create a new function which will take a CSV file shown above as an input to the create_custom_views function

In [14]:
def import_custom_views(filename):
    """
    Function which takes in a csv files as input to the create_custom_views function from the pyhpimc python module
    available at https://github.com/HPNetworking/HP-Intelligent-Management-Center
    :param filename: user-defined filename which contains two column named "name" and "upperview" as input into the
    create_custom_views function from the pyhpimc module.
    :return: returns output of the create_custom_vies function (str) for each item in the CSV file.
    """
    with open(filename) as csvfile:
        # decodes file as csv as a python dictionary
        reader = csv.DictReader(csvfile)
        for view in reader:
            # loads each row of the CSV as a JSON string
            name = view['name']
            upperview = view['upperview']
            if len(upperview) is 0:
                upperview = None
            create_custom_views(auth=auth.creds, url=auth.url,name=name,upperview=upperview)

# Import Custom Views from CSV File

In [15]:
start_time = time.time()
import_custom_views('custom_views.csv')
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.328416109085083 seconds ---


In [16]:
views_list = get_custom_views(url=auth.url, auth=auth.creds)
print_views()
print ("There are a total of " + str(len(views_list)) + " views currently")

There are a total of 103 views currently
Branch1
Branch10
Branch100
Branch11
Branch12
Branch13
Branch14
Branch15
Branch16
Branch17
Branch18
Branch19
Branch2
Branch20
Branch21
Branch22
Branch23
Branch24
Branch25
Branch26
Branch27
Branch28
Branch29
Branch3
Branch30
Branch31
Branch32
Branch33
Branch34
Branch35
Branch36
Branch37
Branch38
Branch39
Branch4
Branch40
Branch41
Branch42
Branch43
Branch44
Branch45
Branch46
Branch47
Branch48
Branch49
Branch5
Branch50
Branch51
Branch52
Branch53
Branch54
Branch55
Branch56
Branch57
Branch58
Branch59
Branch6
Branch60
Branch61
Branch62
Branch63
Branch64
Branch65
Branch66
Branch67
Branch68
Branch69
Branch7
Branch70
Branch71
Branch72
Branch73
Branch74
Branch75
Branch76
Branch77
Branch78
Branch79
Branch8
Branch80
Branch81
Branch82
Branch83
Branch84
Branch85
Branch86
Branch87
Branch88
Branch89
Branch9
Branch90
Branch91
Branch92
Branch93
Branch94
Branch95
Branch96
Branch97
Branch98
Branch99
Branches
My Network View
WAN
{
    "statusDesc": "Unmanaged",
    "

# Cleaning up After Ourselves

In [17]:
def delete_custom_views_csv(filename):
    """
    Function which takes in a csv files as input to the delete_custom_view function from the pyhpeimc python module
    available at https://github.com/HPENetworking/HP-Intelligent-Management-Center
    :param filename: user-defined filename which contains two column named "name" and "upperview" as input into the
    create_custom_views function from the pyhpimc module.
    :return: returns output of the create_custom_vies function (str) for each item in the CSV file.
    """
    with open(filename) as csvfile:
        # decodes file as csv as a python dictionary
        reader = csv.DictReader(csvfile)
        for view in reader:
            # loads each row of the CSV as a JSON string
            name = view['name']
            delete_custom_view(url=auth.url, auth=auth.creds, name=name)

In [18]:
start_time = time.time()
delete_custom_views_csv('custom_views.csv')
print("--- %s seconds ---" % (time.time() - start_time))

--- 31.41563606262207 seconds ---
